# 连接计算结果


In [ ]:
import os
import arcpy
from arcpy.da import *
from arcpy.management import *
import pandas as pd

In [ ]:
# 获取当前日期
def get_dtime():
    # 例如20240516
    import datetime
    dtime = datetime.datetime.now().strftime('%Y%m%d')
    return dtime
# 清理字段
def delete_fields(data_path, fields_to_delete):
    """
    删除指定数据集中的一个或多个字段。

    参数:
    data_path (str): 数据集的路径，可以是要素类、表格等。
    fields_to_delete (list): 需要删除的字段名称列表。
    
    返回:
    None
    """
    try:
        # 检查要删除的字段是否存在于数据集中
        existing_fields = [field.name for field in arcpy.ListFields(data_path)]
        fields_to_delete = [field for field in fields_to_delete if field in existing_fields]
        
        # 删除字段
        if fields_to_delete:
            arcpy.DeleteField_management(data_path, fields_to_delete)
            print(f"成功删除字段: {fields_to_delete}")
        else:
            print("没有找到指定的字段来删除。")
    except Exception as e:
        print(f"删除字段时出错: {e}")



In [ ]:
result_table_path = r"F:\cache_data\zone_ana\dy\prediction_result\prediction_class_WeightedEnsemble_L2_20240619.csv"
shp_path = r"D:\ArcGISProjects\workspace\shbyq\DZ.gdb\pre_sd_data_ele"

In [ ]:
# 允许覆盖
arcpy.env.overwriteOutput =True
# 指定工作空间
arcpy.env.workspace = os.path.dirname(shp_path)

In [ ]:
# 输出列名称限制
arcpy.env.qualifiedFieldNames = False
# 连接表并输出结果
inFeatures = shp_path
joinTable = result_table_path
joinField = "OBJECTID"
out_path = os.path.join(os.path.dirname(inFeatures),f"result_joined_table_{get_dtime()}")
veg_joined_table = AddJoin(in_layer_or_view=inFeatures,in_field=joinField, join_table=joinTable,join_field= joinField)

In [ ]:
CopyFeatures(veg_joined_table, out_path)

In [ ]:
[_.name for _ in arcpy.ListFields(out_path)]

In [ ]:

# 使用示例
fields_to_delete = [ 'Shape_Leng','Shape_Area','XMin','YMin','XMax','YMax','Centroid_X','Centroid_Y','Riverdis','PRE_TZ','OBJECTID_1','DLMC_1','new_TZ_1',]  # 您想要删除的字段列表
delete_fields(out_path, fields_to_delete)

In [ ]:
# 读取土种个数
read_polygon_path = out_path

In [ ]:
read_polygon_df = pd.DataFrame(FeatureClassToNumPyArray(read_polygon_path,['Class1']))

In [ ]:
result_set = pd.unique(read_polygon_df['Class1'])

In [ ]:
result_set,len(result_set)

In [ ]:
# 原土种个数
old_polygon_path = r'C:\Users\Runker\Desktop\train_data\train_data.shp'

In [ ]:
[_.name for _ in arcpy.ListFields(old_polygon_path)]

In [ ]:
old_polygon_df = pd.DataFrame(FeatureClassToNumPyArray(old_polygon_path,['new_TZ']))

In [ ]:
old_set = pd.unique(old_polygon_df['new_TZ'])

In [ ]:
old_set,len(old_set)

In [ ]:
set(old_set)-set(result_set)